# 多入单出的单层神经网络 - 多变量线性回归

- 多样本单特征值计算：
- dataReader：shuffle()
- 前向计算环节:  def __forwardBatch(self, batch_x):
- 损失函数： 
        Z = self.__forwardBatch(X)
        LOSS = (Z - Y)**2
        loss = LOSS.sum()/m/2
- 反向传播：
- 训练环节：

把预测数据放在训练数据中先做归一化，再预测，就不需要还原权重矩阵了
- 归一化
  - 训练样本数据归一化 reader.NormalizeX()
  - 用于预测数据（即特征值）的归一化：reader.NormalizePredicateData()
- 此时loss、w、b的值都没有归一化，不符合神经网络的概率计算优点  
数据结果可视化：def ShowResult(net, reader):  
**可视化不工作？**


In [ ]:


import numpy as np
from pathlib import Path
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm



In [ ]:
file_name = "ch05.npz"

def TargetFunction(x1, x2):
    w1, w2, b = 2, 5, 10
    return w1 * (20 - x1) + w2 * x2 + b


def CreateSampleData(m):
    file = Path(file_name)
    if file.exists():
        data = np.load(file)
        X = data["data"]
        Y = data["label"]
    else:
        X = np.zeros((m, 2))
        # radius [2,20]
        X[:, 0:1] = (np.random.random(1000) * 20 + 2).reshape(1000, 1)
        # [40,120] square
        X[:, 1:2] = np.random.randint(40, 120, (m, 1))
        Y = TargetFunction(X[:, 0:1], X[:, 1:2])
        Noise = np.random.randint(1, 100, (m, 1)) - 50
        Y = Y + Noise
        np.savez(file_name, data=X, label=Y)
    return X, Y

X, Y = CreateSampleData(1000)

print(X[:, 0].max())
print(X[:, 0].min())
print(X[:, 0].mean())
print(X[:, 1].max())
print(X[:, 1].min())
print(X[:, 1].mean())
print(Y.max())
print(Y.min())
print(Y.mean())

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(X[:, 0], X[:, 1], Y)
plt.show()

In [ ]:
class HyperParameters(object):
    def __init__(
        self, input_size, output_size, eta=0.1, max_epoch=1000, batch_size=5, eps=0.1,
    ):
        self.input_size = input_size
        self.output_size = output_size
        self.eta = eta
        self.max_epoch = max_epoch
        self.batch_size = batch_size
        self.eps = eps
        self.NetType = net_type

    def toString(self):
        title = str.format("bz:{0},eta:{1}", self.batch_size, self.eta)
        return title



In [ ]:
class TrainingHistory_1_0(object):
    def __init__(self):
        self.iteration = []
        self.loss_history = []

    def AddLossHistory(self, iteration, loss):
        self.iteration.append(iteration)
        self.loss_history.append(loss)

    def ShowLossHistory(self, hp, xmin=None, xmax=None, ymin=None, ymax=None):
        plt.plot(self.iteration, self.loss_history)
        title = hp.toString()
        plt.title(title)
        plt.xlabel("iteration")
        plt.ylabel("loss")
        if xmin != None and ymin != None:
            plt.axis([xmin, xmax, ymin, ymax])
        plt.show()
        return title

    def GetLast(self):
        count = len(self.loss_history)
        return self.loss_history[count-1], self.w_history[count-1], self.b_history[count-1]

In [ ]:
class NeuralNet(object):
    def __init__(self, hp):
        self.hp = hp
        self.W = np.zeros((self.hp.input_size, self.hp.output_size))#权重矩阵由神经网络输入输出个数决定
        self.B = np.zeros((1, self.hp.output_size))
# 多样本计算
    def __forwardBatch(self, batch_x):
        Z = np.dot(batch_x, self.W) + self.B
        return Z

    def __backwardBatch(self, batch_x, batch_y, batch_z):
        m = batch_x.shape[0]
        dZ = batch_z - batch_y
        dB = dZ.sum(axis=0, keepdims=True)/m
        dW = np.dot(batch_x.T, dZ)/m
        return dW, dB

    def __update(self, dW, dB):
        self.W = self.W - self.hp.eta * dW
        self.B = self.B - self.hp.eta * dB

    def inference(self, x):
        return self.__forwardBatch(x)
    def __checkLoss(self, dataReader):#多样本损失函数计算
        X,Y = dataReader.GetWholeTrainSamples()
        m = X.shape[0]
        Z = self.__forwardBatch(X)
        LOSS = (Z - Y)**2
        loss = LOSS.sum()/m/2
        return loss
    
 
#训练
    def train(self, dataReader, checkpoint):
        # calculate loss to decide the stop condition
        loss_history = TrainingHistory_1_0()
        loss = 10
        if self.hp.batch_size == -1:
            self.hp.batch_size = dataReader.num_train
        max_iteration = math.ceil(dataReader.num_train / self.hp.batch_size)
        checkpoint_iteration = (int)(max_iteration * checkpoint)

        for epoch in range(self.hp.max_epoch):
            print("epoch=%d" %epoch)
            dataReader.Shuffle()
            for iteration in range(max_iteration):
                # get x and y value for one sample
                batch_x, batch_y = dataReader.GetBatchTrainSamples(self.hp.batch_size, iteration)
                # get z from x,y
                batch_z = self.__forwardBatch(batch_x)
                # calculate gradient of w and b
                dW, dB = self.__backwardBatch(batch_x, batch_y, batch_z)
                # update w,b
                self.__update(dW, dB)

                total_iteration = epoch * max_iteration + iteration
                if (total_iteration+1) % checkpoint_iteration == 0:
                    loss = self.__checkLoss(dataReader)
                    print(epoch, iteration, loss, self.W, self.B)
                    loss_history.AddLossHistory(epoch*max_iteration+iteration, loss)
                    if loss < self.hp.eps:
                        break
                    #end if
                #end if
            # end for
            if loss < self.hp.eps:
                break
        # end for
        loss_history.ShowLossHistory(self.hp)
        print("W=", self.W)
        print("B=", self.B)


In [ ]:
class SimpleDataReader(object):
    def __init__(self, data_file):
        self.train_file_name = data_file
        self.num_train = 0
        self.XTrain = None
        self.YTrain = None
        self.X_norm=None
        self.XRAW = None    # raw x
        self.YRaw = None    # raw y
    #样本随机化
    def Shuffle(self):
        seed = np.random.randint(0,100)
        np.random.seed(seed)
        XP = np.random.permutation(self.XTrain)
        np.random.seed(seed)
        YP = np.random.permutation(self.YTrain)
        self.XTrain = XP
        self.YTrain = YP
    # read data from file
    def ReadData(self):
        train_file = Path(self.train_file_name)
        if train_file.exists():
            data = np.load(self.train_file_name)
            self.XRAW = data["data"]
            self.YRAW = data["label"]
            self.num_train = self.XRAW.shape[0]
            self.XTrain = self.XRAW
            self.YTrain = self.YRAW
        else:
            raise Exception("Cannot find train file!!!")
        # end if

    # get batch training data
    def GetSingleTrainSample(self, iteration):
        x = self.XTrain[iteration]
        y = self.YTrain[iteration]
        return x, y
        
    def GetBatchTrainSamples(self, batch_size, iteration):
        start = iteration * batch_size
        end = start + batch_size
        batch_X = self.XTrain[start:end,:]
        batch_Y = self.YTrain[start:end,:]
        return batch_X, batch_Y

    def GetWholeTrainSamples(self):
        return self.XTrain, self.YTrain
    #X_new矩阵存放归一化后的数据，X_norm存放所有特征值归一化时的最小值和range值
    #计算归一化数据，即将X_raw每一列的数据归一化到0-1之间，存放到X_new中
    def NormalizeX(self):
        X_new = np.zeros(self.XRAW.shape)
        num_feature = self.XRAW.shape[1]
        self.X_norm = np.zeros((num_feature,2))
        # 按列归一化,即所有样本的同一特征值分别做归一化
        for i in range(num_feature):
            # get one feature from all examples
            col_i = self.XRAW[:,i]
            max_value = np.max(col_i)
            min_value = np.min(col_i)
            # min value
            self.X_norm[i,0] = min_value 
            # range value
            self.X_norm[i,1] = max_value - min_value 
            new_col = (col_i - self.X_norm[i,0])/(self.X_norm[i,1])
            X_new[:,i] = new_col
        #end for
        self.XTrain = X_new
        return X_new
    
    def NormalizePredicateData(self, X_raw):
        X_new=np.zeros(X_raw.shape)
        n=X_raw.shape[1]
        for i in range(n):
            col_i=X_raw[:,i]
            X_new[:,i]=(col_i-self.X_norm[i,0])/self.X_norm[i,1]
        return X_new



In [ ]:
def ShowResult(net, reader):
    # draw example points
    X,Y = reader.GetWholeTrainSamples()
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(X[:,0],X[:,1],Y)
    # draw fitting surface
    p = np.linspace(0,1)
    q = np.linspace(0,1)
    P,Q = np.meshgrid(p,q)
    R = np.hstack((P.ravel().reshape(2500,1), Q.ravel().reshape(2500,1)))
    Z = net.inference(R)
    Z = Z.reshape(50,50)
    ax.plot_surface(P,Q,Z, cmap='rainbow')
    plt.show()


In [ ]:
#主函数

#file_name = "ch05.npz"
#data
reader = SimpleDataReader(file_name)
reader.ReadData()
reader.NormalizeX()#样本数据归一化
#net
params = HyperParameters(2, 1, eta=0.1, max_epoch=10, batch_size=1, eps = 1e-5)
net = NeuralNet(params)
net.train(reader, checkpoint=0.1)
# inference
x1 = 15
x2 = 93
x=np.array([x1,x2]).reshape(1,2)
x_new=reader.NormalizePredicateData(x)#预测数据归一化
print("Z=", net.inference(x_new))#采用归一化的数据进行预测
ShowResult(net, reader)
